In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import cv2
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm.notebook import trange

In [ ]:
train_data = pd.read_csv('train.csv')
train_data

In [ ]:
X = train_data.drop('label', axis=1).values.reshape(-1,28,28)
y = train_data.label.values
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
len(X_train), len(X_test)

In [ ]:
to_plot = X[:30]
fig = plt.figure(figsize=(20,5))
for i,val in enumerate(to_plot):
    ax = fig.add_subplot(3,10,i+1)
    ax.imshow(val)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
class MLP(nn.Module):
    def __init__(self, inp, h1, out):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(inp, h1)
        self.l2 = nn.ReLU()
        self.l3 = nn.Linear(h1, out)
    def forward(self, x):
        out = self.l1(x)
        out = self.l2(out)
        out = self.l3(out)
        return out

model = MLP(28*28, 100, 10).to(device)

In [ ]:
loss_category = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.00001)
bs = 50000
epochs = 1500

In [ ]:
class MNIST(Dataset):
    def __init__(self, x_, y_):
        self.X = x_
        self.y = y_
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

train_dataset = MNIST(X_train, y_train)
test_dataset = MNIST(X_test, y_test)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)
len(train_dataset), len(test_dataset)

In [ ]:
for epoch in trange(epochs):
    for x_,y_ in train_dataloader:
        x_ = x_.float().to(device).view(-1, 28*28)
        y_ = y_.int().to(device)

        # forward pass
        y_pred = model(x_)
        y_ = F.one_hot(y_.clone().detach().long(), 10).float()

        # calc loss
        loss = loss_category(y_pred, y_)

        # backprop
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
    if (epoch+1)%10 == 0: print(f'Epoch {epoch}: {loss.item():2f}')


In [ ]:
with torch.no_grad():
    score = 0
    ct = 0
    for x_,y_ in train_dataloader:
        x_ = x_.float().to(device).view(-1, 28*28)
        y_ = y_.int().to(device)

        y_pred = model(x_)
        for i,j in zip(y_, torch.argmax(y_pred, dim=1)):
            score += i.item() == j.item()
            ct += 1
    print(f'Train Accuracy {score*100/ ct} %')


In [ ]:
with torch.no_grad():
    score = 0
    ct = 0
    for x_,y_ in test_dataloader:
        x_ = x_.float().to(device).view(-1, 28*28)
        y_ = y_.int().to(device)

        y_pred = model(x_)
        for i,j in zip(y_, torch.argmax(y_pred, dim=1)):
            score += i.item() == j.item()
            ct += 1
    print(f'Test Accuracy {score*100/ ct} %')